In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
time = "2024-12-31_13:33"
time = "2025-01-02_08:33"
time = "2025-01-03_06:00"
time = "2025-01-04_09:44"
time = "2025-01-06_17:52"
time = "2025-01-20_17:15"
time = "2025-01-20_18:26"
time = "2025-01-21_11:28" 
time = "2025-01-26_21:55"


raw_df = pd.read_csv(f"/projects/genomic-ml/da2343/ml_project_2/unsupervised/kmeans/results/{time}_results.csv")
raw_df

,window,train_actual_return,train_num_trades,train_direction,test_actual_return,test_num_trades,test_direction,train_average_return,train_sharpe_ratio,train_profit_factor,...,total_windows,reverse_test,num_clusters,clustering_algorithm,train_period,test_period,random_seed,instrument,num_perceptually_important_points,price_history_length
0,300,0.311739,24,long,-0.059225,20,long,0.130820,1.445717,1000.0,...,101,False,5,kmeans,2,1,426,EXC_M15,4,24
1,301,0.060352,49,short,-0.003620,21,short,0.130820,1.445717,1000.0,...,101,False,5,kmeans,2,1,426,EXC_M15,4,24
2,302,0.092037,54,long,0.216992,19,long,0.130820,1.445717,1000.0,...,101,False,5,kmeans,2,1,426,EXC_M15,4,24
3,303,0.235039,26,long,0.500978,22,long,0.130820,1.445717,1000.0,...,101,False,5,kmeans,2,1,426,EXC_M15,4,24
4,304,0.090538,35,long,-0.032551,12,long,0.130820,1.445717,1000.0,...,101,False,5,kmeans,2,1,426,EXC_M15,4,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4862,397,0.612699,78,long,0.006709,19,long,0.285394,1.252401,1000.0,...,100,False,5,birch,3,1,42,QCOM_M15,4,24
4863,398,0.234791,52,long,0.020676,32,long,0.285394,1.252401,1000.0,...,100,False,5,birch,3,1,42,QCOM_M15,4,24
4864,399,0.129696,59,long,-0.006213,27,long,0.285394,1.252401,1000.0,...,100,False,5,birch,3,1,42,QCOM_M15,4,24
4865,400,0.051924,53,short,0.038907,22,short,0.285394,1.252401,1000.0,...,100,False,5,birch,3,1,42,QCOM_M15,4,24


In [3]:
def analyze_performance(raw_df):
    # Group by relevant parameters and calculate averages
    group_cols = ['instrument', 'num_clusters', 'clustering_algorithm', 'train_period', 
                 'reverse_test', 'random_seed']
    
    # Calculate mean performance metrics for each parameter combination
    results_df = raw_df.groupby(group_cols).agg({
        'test_profit_factor': 'mean',
        'test_sharpe_ratio': 'mean',
        'test_win_ratio': 'mean',
        'test_num_trades': 'mean'
    }).reset_index()
    
    # Calculate combined score
    results_df['combined_score'] = (
        results_df['test_profit_factor'] * 0.4 +
        results_df['test_sharpe_ratio'] * 0.4 +
        (results_df['test_num_trades'] / results_df['test_num_trades'].max()) * 0.2
    )
    
    # Create display DataFrame with proper formatting
    display_df = pd.DataFrame({
        'Instrument': results_df['instrument'],
        'Clusters': results_df['num_clusters'],
        'Algorithm': results_df['clustering_algorithm'],
        'Train Period': results_df['train_period'].astype(str) + 'w',
        'Rev.Test': results_df['reverse_test'],
        'Seed': results_df['random_seed'],
        'PF': results_df['test_profit_factor'].round(3),
        'Sharpe': results_df['test_sharpe_ratio'].round(3),
        'Win%': (results_df['test_win_ratio'] * 100).round(1),
        'Trades': results_df['test_num_trades'].round(1),
        'Score': results_df['combined_score'].round(3)
    })
    
    # Sort by instrument name
    display_df = display_df.sort_values('Instrument')
    
    # Apply styling with color gradients
    styled_df = display_df.style\
        .format({
            'PF': '{:.3f}',
            'Sharpe': '{:.3f}',
            'Win%': '{:.1f}%',
            'Trades': '{:.1f}',
            'Score': '{:.3f}'
        })\
        .background_gradient(subset=['Score'], cmap='YlOrRd')\
        .background_gradient(subset=['PF'], cmap='YlOrRd')\
        .background_gradient(subset=['Sharpe'], cmap='YlOrRd')\
        .background_gradient(subset=['Win%'], cmap='YlOrRd')\
        .background_gradient(subset=['Trades'], cmap='YlOrRd')\
        .set_properties(**{
            'text-align': 'right',
            'font-family': 'monospace',
            'padding': '5px'
        })\
        .hide(axis="index")
    
    return results_df, styled_df

# Example usage:
results_df, styled_df = analyze_performance(raw_df)
styled_df  # Display the styled table

Instrument,Clusters,Algorithm,Train Period,Rev.Test,Seed,PF,Sharpe,Win%,Trades,Score
AAPL_M15,5,birch,1w,False,42,1.155,0.050,50.5%,21.8,0.647
ADI_M15,5,kmeans,1w,False,431,0.827,-0.066,42.5%,21.2,0.465
ADP_M15,5,birch,14w,False,42,0.595,-0.161,51.2%,19.3,0.321
AMD_M15,5,kmeans,7w,False,737,0.949,-0.017,52.6%,24.4,0.557
AMGN_M15,7,kmeans,1w,False,431,0.720,-0.102,53.9%,13.7,0.351
AMZN_M15,7,kmeans,7w,False,426,1.323,0.089,59.8%,17.3,0.696
ASML_M15,7,kmeans,11w,False,737,1.176,0.050,52.5%,15.0,0.604
ATVI_M15,7,birch,9w,False,42,0.823,-0.063,46.1%,16.1,0.426
AVGO_M15,5,birch,5w,False,42,0.858,-0.050,47.5%,19.5,0.471
BKNG_M15,5,birch,14w,False,42,1.391,0.060,48.3%,21.2,0.742


In [7]:
# results_df

# filter where PF is greater than 1.5
filtered_results_df = results_df[results_df['test_profit_factor'] >= 1.1]
filtered_results_df

,instrument,num_clusters,clustering_algorithm,train_period,reverse_test,random_seed,test_profit_factor,test_sharpe_ratio,test_win_ratio,test_num_trades,combined_score
0,AAPL_M15,5,birch,1,False,42,1.155434,0.049560,0.504902,21.774510,0.647262
5,AMZN_M15,7,kmeans,7,False,426,1.323236,0.088988,0.597938,17.268041,0.695951
6,ASML_M15,7,kmeans,11,False,737,1.176149,0.049826,0.524752,15.000000,0.604237
9,BKNG_M15,5,birch,14,False,42,1.391372,0.060431,0.482759,21.218391,0.741765
12,CSCO_M15,5,kmeans,6,False,426,1.864802,0.144499,0.546392,23.195876,0.979773
13,CSX_M15,7,birch,4,False,42,1.526420,0.148220,0.544118,11.352941,0.756023
18,GILD_M15,5,kmeans,11,False,431,1.490351,0.133309,0.550000,21.530000,0.812873
19,GOOGL_M15,7,kmeans,1,False,426,1.420858,0.093210,0.544118,17.352941,0.737333
20,IDXX_M15,7,kmeans,13,False,426,1.624322,0.164523,0.520408,15.102041,0.830160
21,ILMN_M15,7,kmeans,1,False,431,1.417451,0.106964,0.455882,14.245098,0.717884
